### Requirements

In [1]:
# base
import sys
import os
#cleaning
import re
import unicodedata
#manupulate numbers
import numpy as np
import pandas as pd
import random
#Graph
import seaborn  as sns
import matplotlib.pyplot as plt
from datetime import date,datetime,timedelta
#store
import pickle
#gcp
sys.path.append('..')
from settings import Settings
settings = Settings()
from clients.google import Gcp
gcp = Gcp(settings.EMBBEDINGS_BUCKET)
#modeling
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, skm
#ensamble
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier


# data

In [2]:
files = [
    'usem/1697445298_embedded_reviews_train.npz',
    'usem/1697445298_embedded_reviews_test.npz'
    ]

In [ ]:
df = gcp.read_np_from_gcs(files[0], allow_pickle=True)
if df is not None and len(df) > 0:
    df_train = (
        pd.DataFrame({x: df[x] for x in df})
        .rename(columns={"index": "id"})
        .set_index("id")
    )
df = gcp.read_np_from_gcs(files[1], allow_pickle=True)
if df is not None and len(df) > 0:
    df_test = (
        pd.DataFrame({x: df[x] for x in df})
        .rename(columns={"index": "id"})
        .set_index("id")
    )
os.remove("file.npz")

In [4]:
df_train.columns

Index(['embedded_review', 'y'], dtype='object')

In [5]:
df_train

,embedded_review,y
id,,
270032,"[0.051702406, 0.0063196365, -0.0019192067, 0.0...","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
273058,"[0.026936151, -0.07401035, -0.02367169, 0.0012...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
272643,"[0.06126295, -0.005612696, 0.024638325, 0.0139...","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
297102,"[-0.0500299, -0.09700354, 0.01769823, -0.02234...","[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]"
299760,"[0.04312534, -0.032369155, -0.023003675, 0.021...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
...,...,...
267064,"[0.024140796, -0.016030194, -0.032259483, -0.0...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
304503,"[-0.029617317, -0.04365659, 0.021056803, -0.05...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
297491,"[0.061326973, 0.013675633, 0.009585054, -0.001...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [6]:
len(df_train.embedded_review.values[0])

512

In [7]:
len(df_train.y.values[0])

8

In [8]:
df_test.columns

Index(['embedded_review', 'y'], dtype='object')

In [9]:
df_test

,embedded_review,y
id,,
280807,"[-0.019385519, 0.061420172, -0.034199018, 0.06...","[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
225426,"[0.016546639, 0.04594232, 0.035827015, -0.0418...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
258292,"[-0.062319446, 0.017953802, 0.0014271716, 0.02...","[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0]"
307508,"[0.021104338, -0.03740269, -0.04002929, -0.027...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
276805,"[-0.030853122, -0.01589446, -0.005709037, 0.01...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
...,...,...
224886,"[-0.013254025, -0.008982382, -0.014643711, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
261886,"[0.0005017076, -0.038983814, -0.026330085, 0.0...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
296273,"[0.045025535, -0.0063450993, -0.010803862, 0.0...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [10]:
len(df_test.embedded_review.values[0])

512

In [11]:
len(df_test.y.values[0])

8

# Sampling

In [12]:
X_train = np.array(list(df_train['embedded_review']))
X_test = np.array(list(df_test['embedded_review']))
y_train = np.array(list(df_train['y']))
y_test = np.array(list(df_test['y']))

# Modeling

## Random forest

In [13]:
rf=RandomForestClassifier(random_state=2023,
                       n_jobs=-1,
                       class_weight='balanced')
#multilabel_et = OneVsOneClassifier(et)
multilabel_rf = OneVsRestClassifier(rf)

In [14]:
param_grid={
        'estimator__max_features':  ['sqrt', 'log2'],
        'estimator__max_depth' : range(5,25,5),
        'estimator__min_samples_leaf': [0.001,0.1,0.25,0.5],
        'estimator__min_samples_split': range(15,60,5),
        'estimator__criterion' :['gini', 'entropy'],
       'estimator__n_estimators': range(50,200,25),
    }

In [15]:
%%time
CV_rfc = GridSearchCV(
    estimator=multilabel_rf, 
    param_grid=param_grid, 
    scoring='accuracy', 
    cv= 5
)
CV_rfc.fit(X_train, y_train)

CPU times: total: 4d 1h 7min 43s
Wall time: 1d 3h 19min 41s


GridSearchCV(cv=5,
             estimator=OneVsRestClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                                            n_jobs=-1,
                                                                            random_state=2023)),
             param_grid={'estimator__criterion': ['gini', 'entropy'],
                         'estimator__max_depth': range(5, 25, 5),
                         'estimator__max_features': ['sqrt', 'log2'],
                         'estimator__min_samples_leaf': [0.001, 0.1, 0.25, 0.5],
                         'estimator__min_samples_split': range(15, 60, 5),
                         'estimator__n_estimators': range(50, 200, 25)},
             scoring='accuracy')

In [17]:
print("Best: %f using %s" % (CV_rfc.best_score_, CV_rfc.best_params_))

Best: 0.432061 using {'estimator__criterion': 'entropy', 'estimator__max_depth': 10, 'estimator__max_features': 'sqrt', 'estimator__min_samples_leaf': 0.001, 'estimator__min_samples_split': 45, 'estimator__n_estimators': 100}


In [18]:
y_hat = CV_rfc.predict(X_test)
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,y_hat))

Accuracy for Random Forest on CV data:  0.41312422616591005


In [20]:
print("f1 score for Random Forest on CV data: ",f1_score(y_test,y_hat, average='micro'))

f1 score for Random Forest on CV data:  0.612555260831123


In [28]:
print(multilabel_confusion_matrix(y_test, y_hat))

[[[2228   10]
  [ 132   53]]

 [[2005   56]
  [ 199  163]]

 [[1871  133]
  [ 201  218]]

 [[2085   88]
  [ 139  111]]

 [[1541  132]
  [ 325  425]]

 [[2140   48]
  [ 105  130]]

 [[1436  242]
  [ 215  530]]

 [[2155   40]
  [ 126  102]]]


In [23]:
print(classification_report(y_test, y_hat, zero_division=1))

              precision    recall  f1-score   support

           0       0.84      0.29      0.43       185
           1       0.74      0.45      0.56       362
           2       0.62      0.52      0.57       419
           3       0.56      0.44      0.49       250
           4       0.76      0.57      0.65       750
           5       0.73      0.55      0.63       235
           6       0.69      0.71      0.70       745
           7       0.72      0.45      0.55       228

   micro avg       0.70      0.55      0.61      3174
   macro avg       0.71      0.50      0.57      3174
weighted avg       0.71      0.55      0.61      3174
 samples avg       0.79      0.63      0.58      3174



## saving pickle module

In [35]:
timestamp = int(datetime.utcnow().timestamp())
pathfile = f'{timestamp}_rf_tf512.pkl'
print(f' Stored model: {pathfile}')
pickle.dump(CV_rfc, open(pathfile, "wb"))

 Stored model: 1697700992_rf_tf512.pkl


In [36]:
gcp.write_file_gcs(
        pathfile,
        f'models/{pathfile}'
    )

## Validation

### local

In [38]:
pkl_filename_local = '1697700992_rf_tf512.pkl'
with open(pkl_filename_local, 'rb') as file:
    uploaded_CV_rfc_local = pickle.load(file)

In [41]:
y_hat_rfc = uploaded_CV_rfc_local.predict(X_test)
print("Test f1_score: {0:.4f} %".format(100 * f1_score(y_test,y_hat_rfc, average='micro')))

Test f1_score: 61.2555 %


In [42]:
print(classification_report(y_test, y_hat_rfc, zero_division=1))

              precision    recall  f1-score   support

           0       0.84      0.29      0.43       185
           1       0.74      0.45      0.56       362
           2       0.62      0.52      0.57       419
           3       0.56      0.44      0.49       250
           4       0.76      0.57      0.65       750
           5       0.73      0.55      0.63       235
           6       0.69      0.71      0.70       745
           7       0.72      0.45      0.55       228

   micro avg       0.70      0.55      0.61      3174
   macro avg       0.71      0.50      0.57      3174
weighted avg       0.71      0.55      0.61      3174
 samples avg       0.79      0.63      0.58      3174



In [43]:
print(multilabel_confusion_matrix(y_test, y_hat))

[[[2228   10]
  [ 132   53]]

 [[2005   56]
  [ 199  163]]

 [[1871  133]
  [ 201  218]]

 [[2085   88]
  [ 139  111]]

 [[1541  132]
  [ 325  425]]

 [[2140   48]
  [ 105  130]]

 [[1436  242]
  [ 215  530]]

 [[2155   40]
  [ 126  102]]]


### cloud

In [ ]:
uploaded_CV_rfc_cloud = gcp.read_np_from_gcs(f'models/{pathfile}', allow_pickle=True)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bc2cb277-c3fd-470b-bd8b-89f9a40be78d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>